# Data Augmentationについて
> Data augmentation can be viewed as an injection of prior knowledge about the invariant properties of the data against certain transformations. Augmented data can cover unexplored input space, prevent overfitting, and improve the generalization ability of a DL model.

[https://arxiv.org/pdf/1706.00527.pdf](https://arxiv.org/pdf/1706.00527.pdf)より引用

Data Augmentationとは画像でお馴染みの操作で、Deep Learning大流行の昨今、皆さんもイヌネコの画像分類で左右にフリップしたり微小な平行移動や回転を加えてデータ数を水増しするというのを見たことがあるでしょう。この画像の例からData Augmentationはデータの水増しという印象になりがちですが、ではこの操作による精度のゲインはどこから来ているのでしょうか？無から有は生まれませんから、それに相当する情報をどこからか得ていることになります。実はこれは本質的には「とある変換操作がターゲットラベルを変化させない」というPrior Informationをモデルに注入することに相当していると言えます。他にも正則化の一種とみる見方もあり、いずれも同じことですが、この説明はデータと向き合い、どのようなaugmentationがあり得るのかを考える上でとても参考になると思っています。この考え方に立つと、画像以外の様々なデータタイプに対しても適用可能になるでしょう。これは、「何がどうなったらラベルがTrueになるか」などを考えるFeature Engineeringとスタンスは異なりますが近いものがあるようにも感じられますね。


In [ ]:
# ライブラリimportしておきます
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical, Sequence
import cv2

In [ ]:
# データ一覧をDataFrameとして読み込みます
df_train = pd.read_csv('/kaggle/input/data-science-spring-osaka-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/data-science-spring-osaka-2021/test.csv')

In [ ]:
# データの先頭部分をチラッと見ておきましょう
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# センサーデータの方も1個目を見てみましょう
df_sensor = pd.read_csv('/kaggle/input/data-science-spring-osaka-2021/train/train_0000.csv')

In [ ]:
df_sensor.head()

In [ ]:
# センサーデータは数百行程度の可変長データです。
# これを高さがHピクセル、幅が1ピクセル、チャネル数20の画像として読み込むことを考えます。
# 今回のデータは1dなのですがあえて幅1ピクセルの2dとして扱うことによってOpenCVなどの画像関連のライブラリや機能を活用することができます。
def create_image(df, H):
    X = []
    for path in df.file_path:
        arr = pd.read_csv('/kaggle/input/data-science-spring-osaka-2021' + path, index_col=0).values.reshape(1,-1,20)
        arr = cv2.resize(arr.astype(float), (H, 1))
        arr = arr.reshape(-1, H, 1, 20)
        X.append(arr)
    X = np.concatenate(X)
    return X

In [ ]:
# 変換を実行します。
X_train = create_image(df_train, 512)
X_test = create_image(df_test, 512)

In [ ]:
# 今回はData Augmentationだけのご紹介ですが、一応ターゲットも変換しておきます。
y_train = df_train.action_seq.values
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y_train))

In [ ]:
# データのshapeを確認しておきましょう。
X_train.shape, y_train.shape, X_test.shape

In [ ]:
# さて、データに種々の変換操作を施した上でランダムにミニバッチを吐き出してくれるGeneratorを定義しておきます。
# Credit: https://github.com/yu4u/mixup-generator
class EskinGenerator(Sequence):
        'Generates data for Keras'
        def __init__(self, X_train, y_train, batch_size=32, shuffle=True, alpha=.2, datagen=None):
            'Initialization'
            self.batch_size = batch_size
            self.X_train = X_train
            self.y_train = y_train
            self.shuffle = shuffle
            self.on_epoch_end()
            self.alpha= alpha
            self.datagen=datagen
    
        def __len__(self):
            'Denotes the number of batches per epoch'
            return int(np.floor(len(self.X_train) / self.batch_size))
    
        def __getitem__(self, index):
            'Generate one batch of data'
            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
            # Generate data
            X, y = self.__data_generation(indexes)
    
            return X, y
    
        def on_epoch_end(self):
            'Updates indexes after each epoch'
            self.indexes = np.arange(len(self.X_train))
            if self.shuffle == True:
                np.random.shuffle(self.indexes)
        
        def __data_generation(self, batch_ids):
            if self.alpha>0:
                _, h, w, c = self.X_train.shape
                l = np.random.beta(self.alpha, self.alpha, self.batch_size)
                X_l = l.reshape(self.batch_size, 1, 1, 1)
                y_l = l.reshape(self.batch_size, 1)

                X1 = self.X_train[batch_ids]
                X2 = self.X_train[np.flip(batch_ids)] #replaced this with flip
                X = X1 * X_l + X2 * (1 - X_l)
                y1 = self.y_train[batch_ids]
                y2 = self.y_train[np.flip(batch_ids)]
                y = y1 * y_l + y2 * (1 - y_l) 
            else:
                X = self.X_train[batch_ids]
                y = self.y_train[batch_ids]
        
            if self.datagen:
                for i in range(self.batch_size):
                    X[i] = self.datagen.random_transform(X[i])
                    X[i] = self.datagen.standardize(X[i])
        
            return X, y

In [ ]:
# EskinGeneratorに今回のデータをセットします。
# ここで、タイムオフセットはそのタイミングがarbiterary decisionであり、本質的にターゲットの値を変えない（パンチを打つタイミングが少々ずれても、同じコンビネーションという事実を変えない）ことから、height_shiftを入れてみます。
# 他にも色々考えられるでしょう。
datagen = image.ImageDataGenerator(
    height_shift_range=0.1,  
)

training_generator = EskinGenerator(X_train, y_train, alpha=0, batch_size=1, datagen=datagen, shuffle=False)

In [ ]:
# 先頭データの右肘部のセンサー値について、augmentationの結果として出力がどのようになるか見てみましょう。
# よくみると、ランダムな時間方向のシフトが適用されているのがわかりますね。
col = 'ELBOW_R'
col_ix = np.where(df_sensor.columns=='ELBOW_R')[0][0]-1

plt.figure(figsize=[30,10])
for i in range(6):
    plt.subplot(2,3,i+1)
    X, y = training_generator.__getitem__(0)
    plt.title('output %d'%i)
    plt.plot(X[0,:,:,col_ix], label=col)
    plt.xlabel('relative time position')
    plt.ylabel('sensor_output')
    plt.legend()
plt.show()

# それでは引き続きデータとコンペをお楽しみください！

In [ ]:
nan